# Vorverarbeitung
Im Laufe der Datenverarbeitung werden Sie feststellen, dass einige Elemente transformiert werden müssen. In diesem Abschnitt werden daher einige Interaktionen vorgestellt, die möglicherweise hilfreich sind. Zur Veranschaulichung werden erneut Auszüge des Datensatzes [Disney+ Movies and TV Shows](https://www.kaggle.com/code/werador/disney-data-analysis/data) verwendet.

In [ ]:
import pandas as pd

from tui_dsg.datasets import disney_plus_titles_path

In [ ]:
df = pd.read_csv(disney_plus_titles_path)
df.head(10)

## Inhaltsverzeichnis
- [Datumsformate](#Datumsformate)
- [NaN entfernen](#NaN-entfernen)
- [Zellen aufteilen](#Zellen-aufteilen)

## Datumsformate
Eventuell haben Sie direkt festgestellt, dass die Spalte `date_added` im Format *Monat Tag, Jahr* vorliegt, was in Europa eher unüblich ist. Die Ausgabe des Typs der Spalte verrät die Hintergründe.

In [ ]:
df['date_added'].dtype, type(df['date_added'][0])

`'O'` steht für Objekte aus Python und lässt vermuten, dass es sich um einen String handelt. Die Ausgabe des Typs des ersten Elements der Spalte bestätigt die Vorahnung. Probleme ergeben sich dann, wenn Sie mit diesen Datumsangaben rechnen, sie vergleichen oder sortieren. Die folgende Zelle zeigt Ihnen alle Zeilen, in denen die Spalte `date_added` kleiner als `2021` ist.

In [ ]:
df[df['date_added'] < '2021'].head(3)

`<` bezieht sich bei Strings jedoch auf eine alphanumerische Sortierung. Dabei ist jedoch jeder Buchstabe, der den Beginn eines Monats darstellt, größer als die Ziffer $2$ und es gibt keine passenden Zeilen, welche die Bedingung erfüllen.

Durch einen Vergleich mit dem Buchstaben *N* werden folglich alle Angaben aus den Monaten November, Oktober und September gefiltert. Deshalb startet der Zeilenindex erst bei $120$.

In [ ]:
df[df['date_added'] < 'N'].head(3)

Im Normalfall erwartet man jedoch ein anderes Ergebnis. Daten, die kleiner als $2021$ sind, sollten alle Jahre vor $2021$ umfassen. Um mit Datumsangaben zu rechnen, müssen diese jedoch zuerst als Datumsangaben gespeichert werden. Am Einfachsten lässt sich das beim Einlesen des Datensatzes erreichen, indem als Parameter `parse_dates` eine Liste der Spalten übergeben wird, die Datumsangaben enthalten.

In [ ]:
df_dates = pd.read_csv(disney_plus_titles_path, parse_dates=['date_added'])
df_dates.head(3)

Anschließend funktioniert die Filterung wie gewünscht.

In [ ]:
df_dates[df_dates['date_added'] < '2021'].head(3)

## NaN entfernen
In der Spalte `country` fehlen einige Einträge. In der CSV-Datei folgt in einigen Zeilen auf ein Komma direkt ein weiteres Komma, sodass der Wert der entsprechenden Spalte leer ist. Pandas erkennt diese leeren Werte und legt sie als `NaN` ab.

Mit der Funktion `dropna` können Sie NaN-Werte entfernen. Standardmäßig werden Zeilen entfernt.

In [ ]:
df.dropna().head(3)

Mit dem Parameter `subset` kann die Suche auf einzelne Spalten beschränkt werden.

In [ ]:
df.dropna(subset='listed_in').head(3)

Der Parameter `axis` kann verwendet werden, um Spalten mit NaN Werten zu entfernen.

In [ ]:
df.dropna(axis=1).head(3)

Die Funktion `fillna` kann stattdessen verwendet werden, um einen Standardwert einzusetzen.

In [ ]:
filled_df = df.copy()
filled_df['country'] = filled_df['country'].fillna('United States')

filled_df.head(3)

## Zellen aufteilen
Gelegentlich ist es notwendig, Zellen aufzuteilen. Die Spalte `listed_in` gibt alle Genres an, in der ein Eintrag auf der Website von Disney+ gelistet wird. Wird zum Beispiel nach dieser Spalte gruppiert, sind `Drama, Historical`, `Drama` und `Historical` verschiedene Kategorien.

In [ ]:
df.groupby('listed_in').size()

Wünschenswert wäre es jedoch, dass ein Eintrag mit *der* Kategorie `Drama, Historical` in beide Kategorien einzeln aufgenommen wird, statt eine neue Kategorie zu eröffnen. Dazu muss zunächst die Spalte aufgeteilt werden.

Dazu erfolgt zunächst ein Zugriff auf die einzelne Spalte. Die zurückgegebene Series wird als String betrachtet und geteilt. Alle Zeilen, deren Einträge NaN waren, bleiben NaN. Alle anderen Zeilen werden in Listen umgewandelt, die an ihren Kommata geteilt wurden. Pandas erhält auch bei dieser Operation den Index.

In [ ]:
df['listed_in'].str.split(', ')

Die entstehende Series lässt sich anhand des ursprünglichen Index mit dem DataFrame verknüpfen.

In [ ]:
df_split = df.assign(listed_in_lists=df['listed_in'].str.split(', '))
df_split.head(5)

`groupby` lässt sich mit einer Spalte bestehend aus Listen jedoch nicht verwenden. Zunächst soll diese Liste also in mehrere Spalten geteilt werden. Der Parameter `expand` verschiebt die entstehenden Listenelemente in einzelne Spalten.

In [ ]:
df['listed_in'].str.split(', ', expand=True)

Das Ergebnis ist nun keine Series, sondern ein DataFrame. Es kann darauhin mit dem ursprünglichen DataFrame vereinigt werden.

In [ ]:
df_split = pd.merge(df,
                    df['listed_in'].str.split(', ', expand=True),
                    left_index=True, right_index=True)
df_split.head(5)

Über mehrere Spalten ist eine `groupby`-Operation möglich, wobei allerdings einige Spalten einen `None` Eintrag enthalten. Je nach gewünschtem Ergebnis kann dies dazu führen, dass komplizierte Verknüpfungen notwendig werden. Es ist daher ebenfalls möglich, die entstandenen Spalten zu Reihen zu stapeln.

In [ ]:
df['listed_in'].str.split(', ', expand=True).stack()

Sie sehen nun, dass der Index des ursprünglichen DataFrame beibehalten wurde, jedoch ein weiterer Subindex hinzukam. Die Nummerierung entspricht der Spaltennummer im vorhergehenden Beispiel. Zum Verknüpfen mit dem ursprünglichen DataFrame muss jedoch der Subindex entfernt werden.

In [ ]:
df['listed_in'].str.split(', ', expand=True).stack().reset_index(level=[1], drop=True)

Versuchen Sie nun allerdings die Funktion `assign` zu verwenden, wird Pandas mit einem Fehler reagieren, da doppelte Labels im Index vorhanden sind. Benennen Sie daher die Series und wandeln es in ein DataFrame um.

In [ ]:
df['listed_in'].str.split(', ', expand=True).stack().reset_index(level=[1], drop=True).rename('genre').to_frame()

Anschließend lässt sich das Ergebnis mit dem ursprünglichen DataFrame verknüpfen.

In [ ]:
df_split = pd.merge(df,
                    df['listed_in'].str.split(', ', expand=True).stack().reset_index(level=[1], drop=True).rename('genre').to_frame(),
                    left_index=True, right_index=True)
df_split.head(5)

Auch wenn diese Operation aus vielen Einzelschritten zusammengesetzt wurde, lassen sich die Erscheinungen pro Kategorie nun zählen.

In [ ]:
df_split.groupby('genre').size()

Ein Blick auf die Größe der DataFrames lässt jedoch ein Problem erahnen. Jedes Werk, das in mehrere Kategorien eingeordnet wurde, ist nun in genau so vielen Zeilen vertreten. Dabei wurden alle anderen Werte multipliziert und in mehreren Zeilen redundant gespeichert. Insbesondere bei Zeichenketten wie dem Titel oder dem Produktionsland führt dies schnell zu einem hohen Speicherverbrauch.

Die folgende Zelle gibt Informationen zum ursprünglichen und zum aufgeteilten DataFrame zurück. Die Größe hat sich dabei nahezu vervierfacht.

In [ ]:
print('df')
df.info()

print('\n')

print('df_split')
df_split.info()